### Installation and connection to database

In [1]:
import ibm_db

In [2]:
%load_ext sql

In [3]:
%sql ibm_db_sa://cqr93261:kdjqZlTu9uG1BWhP@b0aebb68-94fa-46ec-a1fc-1c999edb6187.c3n41cmd0nqnrk39u98g.databases.appdomain.cloud:31249/bludb?security=SSL

In [4]:
%sql ibm_db_sa://

First verify that the tables are in database and if so, list off the column names:
* CENSUS_DATA
* CHICAGO_PUBLIC_SCHOOLS
* CHICAGO_CRIME_DATA

In [5]:
%sql select syscat.columns from CENSUS_DATA

 * ibm_db_sa://cqr93261:***@b0aebb68-94fa-46ec-a1fc-1c999edb6187.c3n41cmd0nqnrk39u98g.databases.appdomain.cloud:31249/bludb
(ibm_db_dbi.ProgrammingError) ibm_db_dbi::ProgrammingError: SQLNumResultCols failed: [IBM][CLI Driver][DB2/LINUXX8664] SQL0206N  "SYSCAT.COLUMNS" is not valid in the context where it is used.  SQLSTATE=42703\r SQLCODE=-206
[SQL: select syscat.columns from CENSUS_DATA]
(Background on this error at: http://sqlalche.me/e/f405)


### Problem 1: Find the total number of crimes recorded in the CRIME table.

In [6]:
%sql SELECT count(*) from CHICAGO_CRIME_DATA

 * ibm_db_sa://cqr93261:***@b0aebb68-94fa-46ec-a1fc-1c999edb6187.c3n41cmd0nqnrk39u98g.databases.appdomain.cloud:31249/bludb
Done.


1
533


### Problem 2: List community areas with per capita income less than 11000.

In [7]:
%%sql 
SELECT community_area_name FROM chicago_socioeconomic_data 
WHERE per_capita_income_ < 11000;

 * ibm_db_sa://cqr93261:***@b0aebb68-94fa-46ec-a1fc-1c999edb6187.c3n41cmd0nqnrk39u98g.databases.appdomain.cloud:31249/bludb
Done.


community_area_name
West Garfield Park
South Lawndale
Fuller Park
Riverdale


### Problem 3: List all case numbers for crimes involving minors?

In [8]:
%%sql 
SELECT case_number FROM chicago_crime_data 
WHERE description like '%MINOR%'

 * ibm_db_sa://cqr93261:***@b0aebb68-94fa-46ec-a1fc-1c999edb6187.c3n41cmd0nqnrk39u98g.databases.appdomain.cloud:31249/bludb
Done.


case_number
HL266884
HK238408


### Problem 4: List all kidnapping crimes involving a child?(children are not considered minors for the purposes of crime analysis)

In [9]:
%%sql 
SELECT case_number, primary_type, description 
FROM chicago_crime_data 
WHERE description like '%CHILD%' AND primary_type='KIDNAPPING'

 * ibm_db_sa://cqr93261:***@b0aebb68-94fa-46ec-a1fc-1c999edb6187.c3n41cmd0nqnrk39u98g.databases.appdomain.cloud:31249/bludb
Done.


case_number,primary_type,description
HN144152,KIDNAPPING,CHILD ABDUCTION/STRANGER


### Problem 5: What kind of crimes were recorded at schools?

In [10]:
%%sql 
SELECT distinct(primary_type) as "Type of Crime" 
FROM chicago_crime_data 
WHERE location_description like '%SCHOOL%'

 * ibm_db_sa://cqr93261:***@b0aebb68-94fa-46ec-a1fc-1c999edb6187.c3n41cmd0nqnrk39u98g.databases.appdomain.cloud:31249/bludb
Done.


Type of Crime
ASSAULT
BATTERY
CRIMINAL DAMAGE
CRIMINAL TRESPA
NARCOTICS
PUBLIC PEACE VI


### Problem 6: List the average safety score for all types of schools.

In [11]:
%%sql 
select "Elementary, Middle, or High School", 
avg(safety_score) as "Average Safety Score" 
from chicago_public_schools 
group by "Elementary, Middle, or High School"

 * ibm_db_sa://cqr93261:***@b0aebb68-94fa-46ec-a1fc-1c999edb6187.c3n41cmd0nqnrk39u98g.databases.appdomain.cloud:31249/bludb
Done.


"Elementary, Middle, or High School",Average Safety Score
ES,49
HS,49
MS,48


### Problem 7: List 5 community areas with highest % of households below poverty line.

In [12]:
%%sql 
select community_area_name, 
percent_households_below_poverty 
from census_data 
order by percent_households_below_poverty DESC limit 5

 * ibm_db_sa://cqr93261:***@b0aebb68-94fa-46ec-a1fc-1c999edb6187.c3n41cmd0nqnrk39u98g.databases.appdomain.cloud:31249/bludb
Done.


community_area_name,percent_households_below_poverty
Riverdale,56.5
Fuller Park,51.2
Englewood,46.6
North Lawndale,43.1
East Garfield Park,42.4


### Problem 8: Which community area(number) is most crime prone?


In [13]:
%%sql
select community_area_number, 
count(community_area_number) as "Occurrences of Crime" 
from chicago_crime_data group by community_area_number 
order by count(community_area_number) DESC limit 1

 * ibm_db_sa://cqr93261:***@b0aebb68-94fa-46ec-a1fc-1c999edb6187.c3n41cmd0nqnrk39u98g.databases.appdomain.cloud:31249/bludb
Done.


community_area_number,Occurrences of Crime
25,43


### Problem 9: Use a sub-query to find the name of the community area with highest hardship index.

In [14]:
%%sql
select community_area_name from census_data 
where hardship_index = (select max(hardship_index) 
                        from census_data)

 * ibm_db_sa://cqr93261:***@b0aebb68-94fa-46ec-a1fc-1c999edb6187.c3n41cmd0nqnrk39u98g.databases.appdomain.cloud:31249/bludb
Done.


community_area_name
Riverdale


### Problem 10: Use a sub-query to determine the Community Area Name with most number of crimes?

In [15]:
%%sql
select community_area_name from census_data
where community_area_number = (
    select community_area_number from chicago_crime_data
    group by community_area_number
    order by count(community_area_number) DESC
    limit 1)


 * ibm_db_sa://cqr93261:***@b0aebb68-94fa-46ec-a1fc-1c999edb6187.c3n41cmd0nqnrk39u98g.databases.appdomain.cloud:31249/bludb
Done.


community_area_name
Austin
